In [112]:
admin_pk = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"

unit_pk1 = "0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d"

employee_pk1 = "0x7c852118294e51e653712a81e05800f419141751be58f605c371e15141b007a6"
employee_pk2 = "0x47e179ec197488593b187f80a00eb0da91f1b9d0b13f8733639f19c30a34926a"

endpoint_uri="http://127.0.0.1:8545"

In [113]:
import json
from web3 import Web3


# 获取部署的合约地址


# 打开并读取JSON文件
with open('../contract/broadcast/TaxManagement.s.sol/31337/run-latest.json', 'r') as file:
    data = json.load(file)

# 遍历交易记录并查找 contractName 为 RMBToken 的交易
for transaction in data['transactions']:
    contract_name = transaction.get('contractName')
    contract_address = Web3.to_checksum_address(transaction.get('contractAddress'))
    if contract_name == 'TaxManagement':
        taxmgr_contract_address = contract_address


print("TaxManagement contract address: ", taxmgr_contract_address)


# 配置 web3 contract 实例

with open('../contract/out/ERC20.sol/ERC20.json', 'r') as f:
    erc20_json = json.load(f)
    erc20_abi = erc20_json['abi']

with open('../contract/out/TaxManagement.sol/TaxManagement.json', 'r') as f:
    taxmgr_json = json.load(f)
    taxmgr_abi = taxmgr_json['abi']

web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))


contract = web3.eth.contract(address=taxmgr_contract_address, abi=taxmgr_abi)



rmbtk_contract_address = contract.functions.getRMBTokenAddress().call()
taxtk_contract_address = contract.functions.getTaxTokenAddress().call()


print("RMBToken contract address: ", rmbtk_contract_address)
print("TaxToken contract address: ", taxtk_contract_address)

rmbtk_contract = web3.eth.contract(address=rmbtk_contract_address, abi=erc20_abi)
taxtk_contract = web3.eth.contract(address=taxtk_contract_address, abi=erc20_abi)


TaxManagement contract address:  0x0165878A594ca255338adfa4d48449f69242Eb8F
RMBToken contract address:  0x3B02fF1e626Ed7a8fd6eC5299e2C54e1421B626B
TaxToken contract address:  0xBA12646CC07ADBe43F8bD25D83FB628D29C8A762


In [114]:
def submit_txn(private_key, transaction):
    # 签名交易
    signed_txn = web3.eth.account.sign_transaction(transaction, private_key)
    print("signed_txn: ", signed_txn)

    # 发送交易
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print("tx_hash: ", tx_hash)

    # 等待交易完成
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("receipt: ", receipt)

    return receipt

admin_address = web3.eth.account.from_key(admin_pk).address

In [79]:

def get_salary_records_from_blockchain(private_key):

    # 通过私钥得到地址
    PA=web3.eth.account.from_key(private_key)
    account = PA.address
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    print(dir(contract.events.TaxCalculated))

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.TaxCalculated.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block,
        argument_filters={'employeeAddress': account}
    )

    # 获取事件日志
    events = event_filter.get_all_entries()
    print("Events: ", events)

    # 解析事件日志
    salary_data = []
    for event in events:
        print("Event: ", event)
        salary_data.append({
            "salary_date": event['args']['timestamp'],  # 假设事件中有一个时间戳字段
            "gross_salary": event['args']['grossSalary'],  # 假设事件中有税前工资字段
            "net_salary": event['args']['netSalary'],
            "tax_amount": event['args']['taxAmount'],
        })

    print("Salary Data: ", salary_data)

    return salary_data


print(get_salary_records_from_blockchain("0x7c852118294e51e653712a81e05800f419141751be58f605c371e15141b007a6"))

Account:  0x90F79bf6EB2c4f870365E785982E1f101E93b906
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_event_abi', '_get_event_filter_params', '_parse_logs', '_process_get_logs_argument_filters', '_set_up_filter_builder', 'abi', 'address', 'build_filter', 'check_for_forbidden_api_filter_arguments', 'contract_abi', 'create_filter', 'event_name', 'factory', 'get_logs', 'process_log', 'process_receipt', 'w3']
Events:  []
Salary Data:  []
[]


In [115]:
def add_unit(private_key, unit_address):
    # 获取账户
    account =  web3.eth.account.from_key(private_key)

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    transaction = contract.functions.addUnit(unit_address).build_transaction({
        'from': account.address,
        'nonce': nonce,
    })
    print("transaction: ", transaction)

    # 签名交易
    signed_txn = web3.eth.account.sign_transaction(transaction, private_key)
    print("signed_txn: ", signed_txn)

    # 发送交易
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print("tx_hash: ", tx_hash)

    # 等待交易完成
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("receipt: ", receipt)
    return receipt

unit_address = web3.eth.account.from_key(unit_pk1).address

add_unit(admin_pk, unit_address)

transaction:  {'value': 0, 'gas': 106301, 'maxFeePerGas': 1051118808, 'maxPriorityFeePerGas': 1000000000, 'chainId': 31337, 'from': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266', 'nonce': 7, 'to': '0x0165878A594ca255338adfa4d48449f69242Eb8F', 'data': '0xa9f7f24700000000000000000000000070997970c51812dc3a010c7d01b50e0d17dc79c8'}
signed_txn:  SignedTransaction(rawTransaction=HexBytes('0x02f891827a6907843b9aca00843ea6ccd883019f3d940165878a594ca255338adfa4d48449f69242eb8f80a4a9f7f24700000000000000000000000070997970c51812dc3a010c7d01b50e0d17dc79c8c001a059267a296cdef216c68d498e0b2f89f502f73b8bceaa55feb775b5dfcea4c672a02394c9312b14f94cd7fbba1454aa256f50ea1761ac55e25469277f1b99f5dd58'), hash=HexBytes('0x0e0affb2410861dd4bd501693483138d9a976837652f2a572729f7936b601979'), r=40323826842249850996815244646942815012812111636765327157776177783728505144946, s=16093831642591672742010510702517493636202224003175393040383431975973187607896, v=1)
tx_hash:  b"\x0e\n\xff\xb2A\x08a\xddK\xd5\x01i4\x83\x13\x8d\x9

AttributeDict({'type': 2,
 'status': 1,
 'cumulativeGasUsed': 106301,
 'logs': [AttributeDict({'address': '0x3B02fF1e626Ed7a8fd6eC5299e2C54e1421B626B',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000000000000000000000070997970c51812dc3a010c7d01b50e0d17dc79c8')],
   'data': HexBytes('0x00000000000000000000000000000000000000000000000000000000000186a0'),
   'blockHash': HexBytes('0x9cd0b1db8f03d561b0be29b79ffd8500269660f2a104fe092ce26f7056866ea8'),
   'blockNumber': 30,
   'blockTimestamp': '0x664fa354',
   'transactionHash': HexBytes('0x0e0affb2410861dd4bd501693483138d9a976837652f2a572729f7936b601979'),
   'transactionIndex': 0,
   'logIndex': 0,
   'removed': False}),
  AttributeDict({'address': '0x0165878A594ca255338adfa4d48449f69242Eb8F',
   'topics': [HexBytes('0x5110fee5d45252242e961f0282be8170bedfe805a06c1b5289f6c7205fa2cbb6'),
    

In [116]:
def get_units(private_key):
    # 获取账户
    account =  web3.eth.account.from_key(private_key)
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.UnitAdded.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block
    )

    # 获取事件日志
    events = event_filter.get_all_entries()

    for event in events:
        print("Event: ", event)
        print("unit address:", event['args']['unitAddress'])

get_units(admin_pk)

Account:  <eth_account.signers.local.LocalAccount object at 0x7f1b33490790>
Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8'}), 'event': 'UnitAdded', 'logIndex': 1, 'transactionIndex': 0, 'transactionHash': HexBytes('0x0e0affb2410861dd4bd501693483138d9a976837652f2a572729f7936b601979'), 'address': '0x0165878A594ca255338adfa4d48449f69242Eb8F', 'blockHash': HexBytes('0x9cd0b1db8f03d561b0be29b79ffd8500269660f2a104fe092ce26f7056866ea8'), 'blockNumber': 30})
unit address: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8


In [126]:
def add_employee(uni_pk, employee_pk, month_salary):

    account =  web3.eth.account.from_key(uni_pk)

    # print(account.address)
    # return

    employee_address = web3.eth.account.from_key(employee_pk).address

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    transaction = contract.functions.addEmployee(
        employee_address, month_salary).build_transaction({
        'from': account.address,
        'nonce': nonce,
    })
    print("transaction: ", transaction)

    submit_txn(uni_pk, transaction)

    txhash=taxtk_contract.functions.approve(contract_address, 100000000000).transact({'from': employee_address})
    tx_receipt = web3.eth.wait_for_transaction_receipt(txhash)

    txhash=rmbtk_contract.functions.approve(contract_address, 100000000000).transact({'from': employee_address})
    tx_receipt = web3.eth.wait_for_transaction_receipt(txhash)




add_employee(unit_pk1, employee_pk1, 10000)

In [128]:
def get_employee(private_key):
    # 获取账户
    account =  web3.eth.account.from_key(private_key)
    print("Account: ", account)

    # 获取最新区块
    latest_block = web3.eth.block_number

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.EmployeeAdded.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block,
        argument_filters={'unitAddress': account.address}
    )

    # 获取事件日志
    events = event_filter.get_all_entries()

    for event in events:
        print("Event: ", event)
        print("unit address:", event['args']['unitAddress'])
        print("employee address:", event['args']['employeeAddress'])
        print()

get_employee(unit_pk1)

Account:  <eth_account.signers.local.LocalAccount object at 0x7f1b332f7dc0>
Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'employeeAddress': '0x90F79bf6EB2c4f870365E785982E1f101E93b906', 'monthlySalary': 10000}), 'event': 'EmployeeAdded', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x0d55baaa8c0386147af062e744331cd78c191b02bca61842ffcd4501998a52c1'), 'address': '0x0165878A594ca255338adfa4d48449f69242Eb8F', 'blockHash': HexBytes('0x92cd89d1cbecb5a2d887693db4d0a27a6363d0435cfab1ce23864f947b36797e'), 'blockNumber': 31})
unit address: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
employee address: 0x90F79bf6EB2c4f870365E785982E1f101E93b906



In [129]:
def settleTaxes(private_key):

    account =  web3.eth.account.from_key(private_key)

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    transaction = contract.functions.settleTaxes().build_transaction({
        'from': account.address,
        'nonce': nonce,
    })
    print("transaction: ", transaction)

    res = submit_txn(private_key, transaction)
    print(res)

settleTaxes(unit_pk1)

transaction:  {'value': 0, 'gas': 209255, 'maxFeePerGas': 1023561928, 'maxPriorityFeePerGas': 1000000000, 'chainId': 31337, 'from': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'nonce': 26, 'to': '0x0165878A594ca255338adfa4d48449f69242Eb8F', 'data': '0xdfab5dc3'}
signed_txn:  SignedTransaction(rawTransaction=HexBytes('0x02f871827a691a843b9aca00843d0250c883033167940165878a594ca255338adfa4d48449f69242eb8f8084dfab5dc3c001a0ee92b2cb31f33b2c83a53c97ee56c374f46b8f241e38d962e41b34a1c703a5d8a07ac8ae427b9df92bafd8db42b65dcb26dffca76d7d4739574114155661c84ce1'), hash=HexBytes('0x71d39b896badea23dfc71202703bd3181a65c509160fffab30bef21693e2d530'), r=107909651623254308479285943501652315589884336426449465541457968541795457672664, s=55536739636353396168410991789453039147408516192680311072301760216792499244257, v=1)
tx_hash:  b'q\xd3\x9b\x89k\xad\xea#\xdf\xc7\x12\x02p;\xd3\x18\x1ae\xc5\t\x16\x0f\xff\xab0\xbe\xf2\x16\x93\xe2\xd50'
receipt:  AttributeDict({'type': 2, 'status': 1, 'cumulativeGasUsed': 18

In [130]:
def getTaxes(private_key):

    account =  web3.eth.account.from_key(private_key)
    res = contract.functions.getTaxes().call({"from": account.address})
    print("taxes: ", res)

# def getTaxes(private_key):
#     account =  web3.eth.account.from_key(private_key)
#     print("Account: ", account)

#     # 获取最新区块
#     latest_block = web3.eth.block_number

#     # 定义事件过滤器，假设事件名为TaxCalculated
#     event_filter = contract.events.TaxSettlement.create_filter(
#         fromBlock=0,  # 可调整为特定区块范围
#         toBlock=latest_block,
#         argument_filters={'unitAddress': account.address}
#     )

#     # 获取事件日志
#     events = event_filter.get_all_entries()

#     for event in events:
#         print("Event: ", event)
#         print("unit address:", event['args']['unitAddress'])
#         print("totalTaxDue:", event['args']['totalTaxDue'])
#         print()

getTaxes(unit_pk1)

taxes:  745


In [131]:
def payTaxes(private_key, amount):

    account =  web3.eth.account.from_key(private_key)
    print("account", account)
    print("account address", account.address)
    print("amount: ", amount)

    # 构建交易
    nonce = web3.eth.get_transaction_count(account.address)
    try:
        # rmbtk_contract.functions.approve(contract_address, amount).transact({'from': account.address})
        # txhash=taxtk_contract.functions.approve(contract_address, amount).transact({'from': account.address})
        # tx_receipt = web3.eth.wait_for_transaction_receipt(txhash)
        # print(taxtk_contract.functions.allowance(
        #     account.address, contract_address).call({"from": account.address}))
        # print(taxtk_contract.functions.balanceOf(account.address).call({"from": account.address}))
        # print(rmbtk_contract.functions.balanceOf(account.address).call({"from": account.address}))
        # return
        transaction = contract.functions.payTaxes(account.address, amount).build_transaction({
            'from': account.address,
            'nonce': nonce,
        })
    except Exception as e:
        print("err:", e)
        return
    print("transaction: ", transaction)

    submit_txn(private_key, transaction)

payTaxes(unit_pk1, 745)

account <eth_account.signers.local.LocalAccount object at 0x7f1b33104400>
account address 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
amount:  745
transaction:  {'value': 0, 'gas': 115037, 'maxFeePerGas': 1020625788, 'maxPriorityFeePerGas': 1000000000, 'chainId': 31337, 'from': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'nonce': 27, 'to': '0x0165878A594ca255338adfa4d48449f69242Eb8F', 'data': '0x2eab257800000000000000000000000070997970c51812dc3a010c7d01b50e0d17dc79c800000000000000000000000000000000000000000000000000000000000002e9'}
signed_txn:  SignedTransaction(rawTransaction=HexBytes('0x02f8b2827a691b843b9aca00843cd5837c8301c15d940165878a594ca255338adfa4d48449f69242eb8f80b8442eab257800000000000000000000000070997970c51812dc3a010c7d01b50e0d17dc79c800000000000000000000000000000000000000000000000000000000000002e9c001a0182ec49672dc66d5abeefd512e1402f6947ab407defdde37c84ad0c1e78de598a01440517f21d6d8757594a8a2b67a09f7d39ae9b1366ca5b0eba7585d54061d79'), hash=HexBytes('0x2e024a031a11ebaf82f5

In [132]:
def getPays(private_key):
    account =  web3.eth.account.from_key(private_key)
    print("Account: ", account.address)

    # 获取最新区块
    latest_block = web3.eth.block_number

    # 定义事件过滤器，假设事件名为TaxCalculated
    event_filter = contract.events.TaxPayment.create_filter(
        fromBlock=0,  # 可调整为特定区块范围
        toBlock=latest_block,
        argument_filters={'unitAddress': account.address}
    )

    # 获取事件日志
    events = event_filter.get_all_entries()

    for event in events:
        print("Event: ", event)
        print("unit address:", event['args']['unitAddress'])
        print("totalTaxDue:", event['args']['totalTaxDue'])
        print("taxAmount:", event['args']['taxAmount'])
        print()

getPays(unit_pk1)

Account:  0x70997970C51812dc3A010C7d01b50e0d17dc79C8
Event:  AttributeDict({'args': AttributeDict({'unitAddress': '0x70997970C51812dc3A010C7d01b50e0d17dc79C8', 'taxAmount': 745, 'totalTaxDue': 0}), 'event': 'TaxPayment', 'logIndex': 2, 'transactionIndex': 0, 'transactionHash': HexBytes('0x2e024a031a11ebaf82f5ad4148f111cf5c5b33aa5a7906104270e963533a7a94'), 'address': '0x0165878A594ca255338adfa4d48449f69242Eb8F', 'blockHash': HexBytes('0x5c9ad96e6aa7df24b2460c087a36c77da7acf03fa616d3ba673798632aa90dd2'), 'blockNumber': 37})
unit address: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
totalTaxDue: 0
taxAmount: 745



In [133]:
# 自定义错误的签名
error_signature = "ERC20InsufficientAllowance(address,uint256,uint256)"
error_signature_hash = Web3.solidity_keccak(['string'], [error_signature]).hex()

# 假设错误数据如下
error_data = '0xfb8f41b20000000000000000000000000165878a594ca255338adfa4d48449f69242eb8f000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e9'


# 检查错误签名的哈希是否匹配
print("error_signature_hash:", error_signature_hash)
print("error_data[:10]:", error_data[:10])
if error_data[:10] == error_signature_hash[:10]:
    # 提取错误参数部分
    error_params = error_data[10:]

    # 每个参数的偏移量和长度为64字符（32字节）
    owner_address = '0x' + error_params[24:64]
    current_allowance = int(error_params[64:128], 16)
    required_allowance = int(error_params[128:], 16)

    print(f"Error: ERC20InsufficientAllowance")
    print(f"Owner address: {owner_address}")
    print(f"Current allowance: {current_allowance}")
    print(f"Required allowance: {required_allowance}")
else:
    print("Error signature does not match.")


error_signature_hash: 0xfb8f41b23e99d2101d86da76cdfa87dd51c82ed07d3cb62cbc473e469dbc75c3
error_data[:10]: 0xfb8f41b2
Error: ERC20InsufficientAllowance
Owner address: 0x0165878a594ca255338adfa4d48449f69242eb8f
Current allowance: 0
Required allowance: 745
